In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
vehicles = pd.read_csv('data/vehicles.csv', dtype=object)
weekly_gas_all_prices = pd.read_csv('data/GASALLCOVW.csv', index_col=0)
weekly_gas_reg_prices = pd.read_csv('data/GASREGCOVW.csv', index_col=0)
weekly_gas_prem_prices = pd.read_csv('data/GASPRMCOVW.csv', index_col=0)
annually_gas_all_prices = pd.read_csv('data/GASALLCOVA.csv', index_col=0)
annually_gas_reg_prices = pd.read_csv('data/GASREGCOVA.csv', index_col=0)
annually_gas_prem_prices = pd.read_csv('data/GASPRMCOVA.csv', index_col=0)

In [25]:
columns_for_vehicles = ['year',
                        'make',
                        'model',
                        'VClass',
                        'fuelType',
                        'fuelType1',
                        'city08',
                        'highway08',
                        'comb08']
vehicles = vehicles[columns_for_vehicles]
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19


In [37]:
categorical_columns = ['make', 'VClass', 'fuelType', 'fuelType1']
numeric_columns = ['city08', 'highway08', 'comb08']

In [38]:
for col in vehicles.columns:
    if col in categorical_columns:
        vehicles[col] = pd.Categorical(vehicles[col])
    elif col in numeric_columns:
        vehicles[col] = pd.to_numeric(vehicles[col])


In [5]:
for col in vehicles.columns:
    print(col)
    print(vehicles[col].dtype)
    if 'category' == str(vehicles[col].dtype):
        print(vehicles[col])

barrels08
object
barrelsA08
object
charge120
object
charge240
object
city08
object
city08U
object
cityA08
object
cityA08U
object
cityCD
object
cityE
object
cityUF
object
co2
object
co2A
object
co2TailpipeAGpm
object
co2TailpipeGpm
object
comb08
object
comb08U
object
combA08
object
combA08U
object
combE
object
combinedCD
object
combinedUF
object
cylinders
object
displ
object
drive
object
engId
object
eng_dscr
object
feScore
object
fuelCost08
object
fuelCostA08
object
fuelType
object
fuelType1
object
ghgScore
object
ghgScoreA
object
highway08
object
highway08U
object
highwayA08
object
highwayA08U
object
highwayCD
object
highwayE
object
highwayUF
object
hlv
object
hpv
object
id
object
lv2
object
lv4
object
make
object
model
object
mpgData
object
phevBlended
object
pv2
object
pv4
object
range
object
rangeCity
object
rangeCityA
object
rangeHwy
object
rangeHwyA
object
trany
object
UCity
object
UCityA
object
UHighway
object
UHighwayA
object
VClass
object
year
object
youSaveSpend
object
guzzle